2 CNN: 0.98
1 CNN: 0.9615 epoch 3
1 CNN: 0.9782 epoch 50
ABC: 0.9575 batch 100, lr 0.0001, metrics criterion 'CE', epoch 3
ABC: 0.9671 batch 100, lr 0.0001, metrics criterion 'CE', epoch 6
ABC: 0.9768 epoch 50 kernel_per_pixel 8

In [1]:
import torch
import torchvision
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils.tools import dotdict
from driver.driver import ABC_Driver
torch.cuda.is_available()
torch.cuda.set_device(0)
import atd2022

In [6]:
mnist_args = dotdict()
mnist_args.train = dotdict()
mnist_args.predict = dotdict()

mnist_args.name = 'mnist'
mnist_args.model = 'abc'
mnist_args.train.batch_size = 60
mnist_args.predict.batch_size = 100

mnist_args.train_epochs = 10
mnist_args.lr = 0.0001
mnist_args.criterion = 'CE'
mnist_args.use_gpu = True

mnist_args.input_channel = 1
mnist_args.input_height = 28
mnist_args.input_width = 28
mnist_args.kernel_size = 9
mnist_args.knpp = 10
mnist_args.knpp2 = 20

In [7]:
atd_args = dotdict()
atd_args.train = dotdict()

atd_args.name = 'atd'
atd_args.model = 'abc'
atd_args.train.batch_size = 30
atd_args.predict_len = 1
atd_args.history_len = 5

atd_args.train_epochs= 100
atd_args.lr = 0.001
atd_args.criterion = 'L1'
atd_args.use_gpu = True

atd_args.input_channel = atd_args.history_len
atd_args.input_height = 1
atd_args.input_width = 5200
atd_args.kernel_size = 6
atd_args.knpp = 6
atd_args.knpp2 = atd_args.predict_len

In [10]:
args = mnist_args
# data = atd2022.io.read_csv()
data = None
driver = ABC_Driver(args, data)
driver.train()

Use GPU: cuda:None
epoch: 0, train_loss: 1.634727318406105
epoch: 1, train_loss: 1.5403486125469208
epoch: 2, train_loss: 1.5267525876760484
epoch: 3, train_loss: 1.5162291921377182
epoch: 4, train_loss: 1.5078587691783905
epoch: 5, train_loss: 1.5012526593208313
epoch: 6, train_loss: 1.4961491212844849
epoch: 7, train_loss: 1.4922577172517777
epoch: 8, train_loss: 1.4891082199811936
epoch: 9, train_loss: 1.4865124011039734


In [11]:
driver.metric()

0.9737

In [9]:
train_pred = driver.predict(driver.data_loader.train).to(float).round()
train_pred

tensor([[[[ 61.,  58., -31.,  ...,  52., 127.,   4.]]],


        [[[ 61.,  47., -42.,  ...,  21., 131., -12.]]],


        [[[ 41.,  24., -17.,  ...,  38., 120.,   5.]]],


        ...,


        [[[ 58.,  50., -12.,  ...,  84., 213.,  24.]]],


        [[[ 32.,  55., -16.,  ...,  53., 210.,  24.]]],


        [[[ 58.,  51., -32.,  ...,  97., 219.,  16.]]]], dtype=torch.float64)

In [10]:
pred = driver.predict().to(float)
pred

tensor([[[[ 46.9483,  57.5578,  -9.9931,  ...,  97.7333, 254.3347,   6.7233]]]],
       dtype=torch.float64)

In [ ]:
data.mean()